# TAU Airline Environment Usage Demo

This notebook demonstrates how to use the airline environment implementation based on TAU bench. It shows various airline reservation operations including booking, searching, updating, and canceling reservations.

## 1. Environment Setup and Initialization

In [6]:
# Import the airline environment
import sys
import os
sys.path.append('/mnt/raid0/home/jiaao/MagicData/Genflow/pipelines/tasks/tool_use/tau_airline')

from airline_env import get_airline_environment, AirlineTools
import json
from pprint import pprint

In [7]:
# Initialize the airline environment
env = get_airline_environment()
tools = env.get_tools()

print("Environment Statistics:")
pprint(env.get_statistics())

Environment Statistics:
{'num_flights': 300,
 'num_flights_instances': 9000,
 'num_reservations': 2000,
 'num_users': 500}


## 2. Exploring Available Data

### 2.1 List All Airports

In [9]:
# Get all available airports
airports = tools.list_all_airports()
print("Available Airports:")
for airport in airports:
    print(f"  {airport.iata}: {airport.city}")

Available Airports:
  SFO: San Francisco
  JFK: New York
  LAX: Los Angeles
  ORD: Chicago
  DFW: Dallas
  DEN: Denver
  SEA: Seattle
  ATL: Atlanta
  MIA: Miami
  BOS: Boston
  PHX: Phoenix
  IAH: Houston
  LAS: Las Vegas
  MCO: Orlando
  EWR: Newark
  CLT: Charlotte
  MSP: Minneapolis
  DTW: Detroit
  PHL: Philadelphia
  LGA: LaGuardia


### 2.2 Explore Sample User Data

In [10]:
# Get a sample user
user_ids = list(env.db.users.keys())[:5]
print(f"Sample user IDs: {user_ids}")

# Get details for the first user
sample_user_id = user_ids[0]
user = tools.get_user_details(sample_user_id)
print(f"\nUser Details for {sample_user_id}:")
print(f"Name: {user.name.first_name} {user.name.last_name}")
print(f"Email: {user.email}")
print(f"Membership: {user.membership}")
print(f"Number of reservations: {len(user.reservations)}")
print(f"Payment methods: {list(user.payment_methods.keys())}")

Sample user IDs: ['mia_li_3668', 'mei_hernandez_8984', 'aarav_nguyen_1055', 'chen_hernandez_2608', 'lucas_hernandez_8985']

User Details for mia_li_3668:
Name: Mia Li
Email: mia.li3818@example.com
Membership: gold
Number of reservations: 3
Payment methods: ['credit_card_4421486', 'certificate_4856383', 'certificate_7504069', 'credit_card_1955700']


### 2.3 Explore Sample Reservation Data

In [11]:
# Get a sample reservation
reservation_ids = list(env.db.reservations.keys())[:3]
print(f"Sample reservation IDs: {reservation_ids}")

# Get details for the first reservation
sample_reservation_id = reservation_ids[0]
reservation = tools.get_reservation_details(sample_reservation_id)
print(f"\nReservation Details for {sample_reservation_id}:")
print(f"User ID: {reservation.user_id}")
print(f"Route: {reservation.origin} → {reservation.destination}")
print(f"Flight Type: {reservation.flight_type}")
print(f"Cabin: {reservation.cabin}")
print(f"Number of passengers: {len(reservation.passengers)}")
print(f"Number of flights: {len(reservation.flights)}")
print(f"Total baggages: {reservation.total_baggages}")
print(f"Insurance: {reservation.insurance}")
print(f"Status: {reservation.status or 'Active'}")

Sample reservation IDs: ['4WQ150', 'VAAOXJ', 'PGAGLM']

Reservation Details for 4WQ150:
User ID: chen_jackson_3290
Route: DFW → LAX
Flight Type: round_trip
Cabin: business
Number of passengers: 3
Number of flights: 2
Total baggages: 5
Insurance: no
Status: Active


## 3. Flight Search Operations

### 3.1 Search for Direct Flights

In [20]:
# Search for direct flights from JFK to LAX on a specific date
origin = "JFK"
destination = "SFO"
date = "2024-05-16"

print(f"Searching for direct flights from {origin} to {destination} on {date}:")
direct_flights = tools.search_direct_flight(origin, destination, date)

if direct_flights:
    for i, flight in enumerate(direct_flights, 1):
        print(f"\nFlight {i}: {flight.flight_number}")
        print(f"  Departure: {flight.scheduled_departure_time_est}")
        print(f"  Arrival: {flight.scheduled_arrival_time_est}")
        print(f"  Available seats: {flight.available_seats}")
        print(f"  Prices: {flight.prices}")
else:
    print("No direct flights found.")

Searching for direct flights from JFK to SFO on 2024-05-16:

Flight 1: HAT023
  Departure: 14:00:00
  Arrival: 20:00:00
  Available seats: {'basic_economy': 9, 'economy': 8, 'business': 11}
  Prices: {'basic_economy': 72, 'economy': 193, 'business': 221}


### 3.2 Search for One-Stop Flights

In [21]:
# Search for one-stop flights
print(f"\nSearching for one-stop flights from {origin} to {destination} on {date}:")
onestop_flights = tools.search_onestop_flight(origin, destination, date)

if onestop_flights:
    for i, (flight1, flight2) in enumerate(onestop_flights[:3], 1):  # Show first 3
        print(f"\nOption {i}:")
        print(f"  First leg: {flight1.flight_number} ({flight1.origin} → {flight1.destination})")
        print(f"    Departure: {flight1.scheduled_departure_time_est}")
        print(f"    Arrival: {flight1.scheduled_arrival_time_est}")
        print(f"  Second leg: {flight2.flight_number} ({flight2.origin} → {flight2.destination})")
        print(f"    Departure: {flight2.scheduled_departure_time_est}")
        print(f"    Arrival: {flight2.scheduled_arrival_time_est}")
else:
    print("No one-stop flights found.")


Searching for one-stop flights from JFK to SFO on 2024-05-16:

Option 1:
  First leg: HAT069 (JFK → SEA)
    Departure: 06:00:00
    Arrival: 12:00:00
  Second leg: HAT011 (SEA → SFO)
    Departure: 13:00:00
    Arrival: 15:00:00

Option 2:
  First leg: HAT069 (JFK → SEA)
    Departure: 06:00:00
    Arrival: 12:00:00
  Second leg: HAT258 (SEA → SFO)
    Departure: 17:00:00
    Arrival: 19:00:00

Option 3:
  First leg: HAT069 (JFK → SEA)
    Departure: 06:00:00
    Arrival: 12:00:00
  Second leg: HAT274 (SEA → SFO)
    Departure: 20:00:00
    Arrival: 22:00:00


### 3.3 Check Flight Status

In [22]:
# Check status of a specific flight
flight_number = "HAT001"
check_date = "2024-05-20"

try:
    status = tools.get_flight_status(flight_number, check_date)
    print(f"Flight {flight_number} on {check_date}: {status}")
except ValueError as e:
    print(f"Error: {e}")

Flight HAT001 on 2024-05-20: available


## 4. Making a New Booking

In [23]:
# Let's find a user with available payment methods
sample_user = None
for user_id in user_ids:
    user = tools.get_user_details(user_id)
    if user.payment_methods:
        sample_user = user
        break

if sample_user:
    print(f"Selected user: {sample_user.user_id} ({sample_user.name.first_name} {sample_user.name.last_name})")
    print(f"Available payment methods: {list(sample_user.payment_methods.keys())}")
    
    # Get a payment method with sufficient balance
    selected_payment = None
    for payment_id, payment_method in sample_user.payment_methods.items():
        if hasattr(payment_method, 'amount') and payment_method.amount > 500:
            selected_payment = payment_id
            print(f"Selected payment method: {payment_id} (Balance: ${payment_method.amount})")
            break
else:
    print("No suitable user found for booking demo.")

Selected user: mia_li_3668 (Mia Li)
Available payment methods: ['credit_card_4421486', 'certificate_4856383', 'certificate_7504069', 'credit_card_1955700']


In [26]:
user

User(user_id='mia_li_3668', name=Name(first_name='Mia', last_name='Li'), address=Address(address1='975 Sunset Drive', address2='Suite 217', city='Austin', country='USA', state='TX', zip='78750'), email='mia.li3818@example.com', dob='1990-04-05', payment_methods={'credit_card_4421486': CreditCard(source='credit_card', id='credit_card_4421486', brand='visa', last_four='7447'), 'certificate_4856383': Certificate(source='certificate', id='certificate_4856383', amount=100.0), 'certificate_7504069': Certificate(source='certificate', id='certificate_7504069', amount=250.0), 'credit_card_1955700': CreditCard(source='credit_card', id='credit_card_1955700', brand='visa', last_four='1907')}, saved_passengers=[Passenger(first_name='Amelia', last_name='Ahmed', dob='1957-03-21')], membership='gold', reservations=['NO6JO3', 'AIXC49', 'HKEG34'])

In [24]:
# Find available flights for booking
booking_origin = "DEN"
booking_destination = "MIA"
booking_date = "2024-05-25"

available_flights = tools.search_direct_flight(booking_origin, booking_destination, booking_date)
print(f"Available flights from {booking_origin} to {booking_destination} on {booking_date}:")

if available_flights:
    selected_flight = available_flights[0]
    print(f"Selected flight: {selected_flight.flight_number}")
    print(f"Available economy seats: {selected_flight.available_seats.get('economy', 0)}")
    print(f"Economy price: ${selected_flight.prices.get('economy', 0)}")
else:
    print("No available flights found for booking.")

Available flights from DEN to MIA on 2024-05-25:
Selected flight: HAT130
Available economy seats: 4
Economy price: $153


In [30]:
selected_payment = 'credit_card_4421486'

In [28]:
available_flights[0]

DirectFlight(flight_number='HAT130', origin='DEN', destination='MIA', status='available', scheduled_departure_time_est='02:00:00', scheduled_arrival_time_est='06:00:00', date=None, available_seats={'basic_economy': 3, 'economy': 4, 'business': 6}, prices={'basic_economy': 53, 'economy': 153, 'business': 460})

In [31]:
# Make a booking (if we have suitable data)
if sample_user and selected_payment and available_flights:
    try:
        # Prepare booking data
        flight_info = {
            "flight_number": selected_flight.flight_number,
            "date": booking_date
        }
        
        passenger_info = {
            "first_name": sample_user.name.first_name,
            "last_name": sample_user.name.last_name,
            "dob": sample_user.dob
        }
        
        economy_price = selected_flight.prices.get('economy', 0)
        payment_info = {
            "payment_id": selected_payment,
            "amount": economy_price
        }
        
        print(f"\nAttempting to book:")
        print(f"Flight: {flight_info}")
        print(f"Passenger: {passenger_info}")
        print(f"Payment: {payment_info}")
        
        # Make the booking
        new_reservation = tools.book_reservation(
            user_id=sample_user.user_id,
            origin=booking_origin,
            destination=booking_destination,
            flight_type="one_way",
            cabin="economy",
            flights=[flight_info],
            passengers=[passenger_info],
            payment_methods=[payment_info],
            total_baggages=1,
            nonfree_baggages=0,
            insurance="no"
        )
        
        print(f"\n✅ Booking successful!")
        print(f"Reservation ID: {new_reservation.reservation_id}")
        print(f"Flight: {new_reservation.flights[0].flight_number}")
        print(f"Total cost: ${new_reservation.flights[0].price}")
        
    except Exception as e:
        print(f"❌ Booking failed: {e}")
else:
    print("Cannot demonstrate booking - missing required data.")


Attempting to book:
Flight: {'flight_number': 'HAT130', 'date': '2024-05-25'}
Passenger: {'first_name': 'Mia', 'last_name': 'Li', 'dob': '1990-04-05'}
Payment: {'payment_id': 'credit_card_4421486', 'amount': 153}

✅ Booking successful!
Reservation ID: HATHAT
Flight: HAT130
Total cost: $153


## 5. Reservation Management Operations

### 5.1 Update Passenger Information

In [32]:
# Get a reservation to update
if 'new_reservation' in locals():
    reservation_to_update = new_reservation.reservation_id
else:
    # Use an existing reservation
    reservation_to_update = sample_reservation_id

print(f"Updating passenger information for reservation: {reservation_to_update}")

try:
    # Get current reservation details
    current_reservation = tools.get_reservation_details(reservation_to_update)
    print(f"Current passengers: {len(current_reservation.passengers)}")
    
    # Update passenger details (keeping same structure)
    updated_passengers = []
    for passenger in current_reservation.passengers:
        updated_passenger = {
            "first_name": passenger.first_name,
            "last_name": passenger.last_name,
            "dob": passenger.dob
        }
        updated_passengers.append(updated_passenger)
    
    # Update the reservation
    updated_reservation = tools.update_reservation_passengers(
        reservation_id=reservation_to_update,
        passengers=updated_passengers
    )
    
    print(f"✅ Passenger information updated successfully!")
    
except Exception as e:
    print(f"❌ Update failed: {e}")

2025-08-20 22:17:04.808 | INFO     | airline_env.tools:update_reservation_passengers:722 - 1
2025-08-20 22:17:04.809 | INFO     | airline_env.tools:update_reservation_passengers:723 - 1


Updating passenger information for reservation: HATHAT
Current passengers: 1
✅ Passenger information updated successfully!


### 5.2 Update Baggage Information

In [33]:
# Update baggage information
try:
    current_reservation = tools.get_reservation_details(reservation_to_update)
    user = tools.get_user_details(current_reservation.user_id)
    
    # Get a payment method for the baggage fee
    payment_method_id = list(user.payment_methods.keys())[0]
    
    print(f"Current baggage: {current_reservation.total_baggages} total, {current_reservation.nonfree_baggages} paid")
    print(f"Adding 1 additional paid bag...")
    
    updated_reservation = tools.update_reservation_baggages(
        reservation_id=reservation_to_update,
        total_baggages=current_reservation.total_baggages + 1,
        nonfree_baggages=current_reservation.nonfree_baggages + 1,
        payment_id=payment_method_id
    )
    
    print(f"✅ Baggage updated successfully!")
    print(f"New baggage: {updated_reservation.total_baggages} total, {updated_reservation.nonfree_baggages} paid")
    print(f"Additional charge: $50")
    
except Exception as e:
    print(f"❌ Baggage update failed: {e}")

Current baggage: 1 total, 0 paid
Adding 1 additional paid bag...
✅ Baggage updated successfully!
New baggage: 2 total, 1 paid
Additional charge: $50


### 5.3 Send Certificate (Compensation)

In [34]:
# Send a certificate to a user as compensation
try:
    user_for_certificate = sample_user.user_id
    certificate_amount = 100
    
    print(f"Sending ${certificate_amount} certificate to user {user_for_certificate}...")
    
    result = tools.send_certificate(
        user_id=user_for_certificate,
        amount=certificate_amount
    )
    
    print(f"✅ {result}")
    
    # Verify the certificate was added
    updated_user = tools.get_user_details(user_for_certificate)
    certificates = [pm for pm_id, pm in updated_user.payment_methods.items() if pm.source == "certificate"]
    print(f"User now has {len(certificates)} certificate(s)")
    
except Exception as e:
    print(f"❌ Certificate sending failed: {e}")

Sending $100 certificate to user mia_li_3668...
✅ Certificate certificate_3221322 added to user mia_li_3668 with amount 100.
User now has 3 certificate(s)


## 6. Cancellation Operations

In [35]:
# Cancel a reservation
if 'new_reservation' in locals():
    reservation_to_cancel = new_reservation.reservation_id
    print(f"Cancelling newly created reservation: {reservation_to_cancel}")
else:
    # Find a non-cancelled reservation to cancel
    reservation_to_cancel = None
    for res_id in list(env.db.reservations.keys())[:5]:
        res = tools.get_reservation_details(res_id)
        if res.status != "cancelled":
            reservation_to_cancel = res_id
            break
    
    if reservation_to_cancel:
        print(f"Cancelling existing reservation: {reservation_to_cancel}")
    else:
        print("No suitable reservation found for cancellation demo.")

if reservation_to_cancel:
    try:
        # Get reservation details before cancellation
        before_cancellation = tools.get_reservation_details(reservation_to_cancel)
        print(f"Before cancellation - Status: {before_cancellation.status or 'Active'}")
        print(f"Payment history entries: {len(before_cancellation.payment_history)}")
        
        # Cancel the reservation
        cancelled_reservation = tools.cancel_reservation(reservation_to_cancel)
        
        print(f"\n✅ Reservation cancelled successfully!")
        print(f"After cancellation - Status: {cancelled_reservation.status}")
        print(f"Payment history entries: {len(cancelled_reservation.payment_history)}")
        print(f"Refund entries added for original payments")
        
    except Exception as e:
        print(f"❌ Cancellation failed: {e}")

2025-08-20 22:17:22.541 | DEBUG    | airline_env.tools:cancel_reservation:389 - {
    "reservation_id": "HATHAT",
    "user_id": "mia_li_3668",
    "origin": "DEN",
    "destination": "MIA",
    "flight_type": "one_way",
    "cabin": "economy",
    "flights": [
        {
            "flight_number": "HAT130",
            "origin": "DEN",
            "destination": "MIA",
            "date": "2024-05-25",
            "price": 153
        }
    ],
    "passengers": [
        {
            "first_name": "Mia",
            "last_name": "Li",
            "dob": "1990-04-05"
        }
    ],
    "payment_history": [
        {
            "payment_id": "credit_card_4421486",
            "amount": 153
        },
        {
            "payment_id": "credit_card_4421486",
            "amount": 50
        }
    ],
    "created_at": "2024-05-15T15:00:00",
    "total_baggages": 2,
    "nonfree_baggages": 1,
    "insurance": "no",
    "status": null
}
2025-08-20 22:17:22.544 | DEBUG    | airline_env

Cancelling newly created reservation: HATHAT
Before cancellation - Status: Active
Payment history entries: 2

✅ Reservation cancelled successfully!
After cancellation - Status: cancelled
Payment history entries: 4
Refund entries added for original payments


## 7. Utility Operations

### 7.1 Mathematical Calculations

In [36]:
# Test the calculate function
calculations = [
    "299 + 50",  # Flight price + baggage
    "399 * 2",   # Flight price for 2 passengers
    "(299 + 50) * 2 + 30",  # Complex calculation
    "1000 - 250",  # Refund calculation
]

print("Testing calculation functionality:")
for expr in calculations:
    try:
        result = tools.calculate(expr)
        print(f"  {expr} = {result}")
    except Exception as e:
        print(f"  {expr} → Error: {e}")

Testing calculation functionality:
  299 + 50 = 349.0
  399 * 2 = 798.0
  (299 + 50) * 2 + 30 = 728.0
  1000 - 250 = 750.0


### 7.2 Human Agent Transfer

In [37]:
# Demonstrate transfer to human agent
summary = "Customer is requesting a complex multi-city itinerary change that requires manual review."
transfer_result = tools.transfer_to_human_agents(summary)
print(f"Transfer request: {summary}")
print(f"Result: {transfer_result}")

Transfer request: Customer is requesting a complex multi-city itinerary change that requires manual review.
Result: Transfer successful


## 8. Error Handling and Edge Cases

In [38]:
# Test various error conditions
print("Testing error handling:")

# 1. Non-existent user
try:
    tools.get_user_details("non_existent_user")
except ValueError as e:
    print(f"✅ Non-existent user error: {e}")

# 2. Non-existent reservation
try:
    tools.get_reservation_details("INVALID_ID")
except ValueError as e:
    print(f"✅ Non-existent reservation error: {e}")

# 3. Non-existent flight
try:
    tools.get_flight_status("INVALID_FLIGHT", "2024-05-20")
except ValueError as e:
    print(f"✅ Non-existent flight error: {e}")

# 4. Invalid calculation
try:
    tools.calculate("invalid expression")
except ValueError as e:
    print(f"✅ Invalid calculation error: {e}")

Testing error handling:
✅ Non-existent user error: User non_existent_user not found
✅ Non-existent reservation error: Reservation INVALID_ID not found
✅ Non-existent flight error: Flight INVALID_FLIGHT not found
✅ Invalid calculation error: Invalid characters in expression


## 9. Environment State Management

In [39]:
# Save current state
print("Saving environment state...")
env.save_state()
print("✅ State saved successfully!")

# Show final statistics
print("\nFinal Environment Statistics:")
pprint(env.get_statistics())

# Reset environment (optional)
# env.reset()
# print("\nEnvironment reset to initial state.")

Saving environment state...
✅ State saved successfully!

Final Environment Statistics:
{'num_flights': 300,
 'num_flights_instances': 9000,
 'num_reservations': 2001,
 'num_users': 500}


## Summary

This notebook demonstrated the comprehensive functionality of the TAU airline environment implementation, including:

1. **Environment Setup**: Initializing the airline environment and exploring the database
2. **Data Exploration**: Browsing airports, users, and reservations
3. **Flight Search**: Finding direct and one-stop flights
4. **Booking Operations**: Creating new reservations with proper validation
5. **Reservation Management**: Updating passenger info, baggage, and sending certificates
6. **Cancellations**: Cancelling reservations with proper refund handling
7. **Utility Functions**: Mathematical calculations and human agent transfers
8. **Error Handling**: Proper error responses for invalid operations
9. **State Management**: Saving and resetting the environment

The implementation closely follows the TAU bench specification and provides a robust foundation for airline reservation system interactions in tool use scenarios.